# Functional Design Patterns
In functional languages, most design patterns are surprisingly around functions.
Although the obvious way was to show examples in functional notation, I chose object (dot) notation and used classes in Python for the extra challenge.

These examples were enspired by OSlash library, Douglas Crockford, Computerphile and MPJ's wonderful youtube channel "Fun Fun Functions"

## Functions as first class citizens
In Functional languages, functions are first citizens. This means they are treated the same way like any other value. They can be assigned to variables, they can be passed as arguments and they can be created inside other functions as well.

In [1]:
def foo():
    return "hello world"

bar = foo
print(bar())

hello world


## Higher order functions

One of the great benefits of that is the ability to write higher order functions. These functions either get a function as a parameter or return a new function.

In [2]:
def double_func(foo):
    return lambda v: 2*foo(v)

def plus1(v):
    return v+1

def plus10(v):
    return v+10

my_func1 = double_func(plus1)
my_func2 = double_func(plus10)
print(my_func1(1), my_func2(1))

4 22


## Safe divider exmple

In [3]:
class Div:
    def __init__(self, value1, value2):
        self.value1 = value1
        self.value2 = value2
    def evaluate(self):
        e1 = self.value1.evaluate() if type(self.value1) == Div else self.value1
        e2 = self.value2.evaluate() if type(self.value2) == Div else self.value2
        
        return e1//e2

In [4]:
print(Div(10,Div(4,Div(2,1))).evaluate())

5


In [5]:
print(Div(10,Div(4,Div(2,0))).evaluate())

ZeroDivisionError: integer division or modulo by zero

In [6]:
class SafeDiv:
    def __init__(self, value1, value2=1):
        self.value1 = value1
        self.value2 = value2
    
    def div(self, v1, v2):
        if v2 == 0:
            print("division by zero", self)
            return None
        return v1 // v2
    
    def evaluate(self):
        e1 = self.value1.evaluate() if type(self.value1) == SafeDiv else self.value1
        e2 = self.value2.evaluate() if type(self.value2) == SafeDiv else self.value2

        if e1 == None or e2 == None:
            return None
        
        return self.div(e1,e2)

    def __str__(self):
        return "SafeDiv(%s, %s)" % (self.value1, self.value2)

In [7]:
print(SafeDiv(10,SafeDiv(SafeDiv(2,1), 0)).evaluate())

division by zero SafeDiv(SafeDiv(2, 1), 0)
None


So much clutter - just because we wanted a safe version for division

# At first there were values...

In [8]:
x = 3
y = 3
print(x == y)

True


values are boring, so sometimes we want to have box.

In [9]:
class Box:
    def __init__(self, v):
        self.value = v
    def __str__(self):
        print("Box(%s)" % self.value)
x = Box(3)
y = Box(3)
print(x == y)

False


Boxes are contexts, they encapsulate the values.

# Lets talk about Arrays...

In [10]:
class Array:
    def __init__(self, *args):
        self.values = [*args]
    def map(self, func):
        return Array(*list(map(func, self.values)))
    def __str__(self):
        return self.values.__str__()

arr = Array(1,2,3)
print(arr)
arr2 = arr.map(lambda v: v*2)
print(arr2)

[1, 2, 3]
[2, 4, 6]


I've implemented a strange wrap on Python list type so the map method would be a method, instead of a global function.
The difference between calling map(lambda, l) and arr.map(lambda) is the notation - first is functional and the other is methodical.

This is just to be consice with the other examples.

As you can see, Arrays simply compose values, and also have a map method.
the map method receives a callback function, which will get one value at a time.

Thus Arrays are Functors.

# Functors - Objects with a map function

Functors are boxes with a "map" method. The idea behind them is that they encapsulate the value, and allows you via the map function access to those values, manipulate them and pack them in a Functor.

In [11]:
class Functor:
    def __init__(self, value):
        self.value = value
    def map(self, func):
        return Functor(func(self.value))
    def __str__(self):
        return "Functor(%s)" % self.value

In [12]:
Five = Functor(5)
print(Five)
Ten = Five.map(lambda v: v*2)
print(Ten)

Functor(5)
Functor(10)


# Functors are very useful, we use them daily. But can we do better?

For instance, what will happen if our callback function will return a Functor?

In [13]:
BadArray = arr.map(lambda v: Array(v,v))
print(BadArray)
BadTen = Five.map(lambda v: Functor(v*2))
print(BadTen)

[<__main__.Array object at 0x7f4eec98ef40>, <__main__.Array object at 0x7f4eec98ebb0>, <__main__.Array object at 0x7f4eec98eb50>]
Functor(Functor(10))


This makes me :(

# Monads

### The Monad Curse
Once you understand monads, you become incapable of explaining what they are.

### definitions
1. In functional programming, a monad is a design pattern that allows structuring programs generically while automating away boilerplate code needed by the program logic - wikipedia

2. A monad is a functor with a flat_map method.

M - is a monad
monad.flat_map(f) - is a higher order function that returns a monad. _f_ is a function that returns a monad.

Python have the OSlash library with implementations of various monads. One of these monads is the Either monad which can help us manage error handling. It has a shortcut to the flat_map method using the pipe (|) operator

In [14]:
from oslash import Right, Left


class EitherDiv:
    def __init__(self, value1, value2=1):
        self.value1 = value1
        self.value2 = value2
    
    def div(self, v1, v2):
        if(v2 == 0):
            return Left("(division by zero) %s" % self)
        return Right(v1 // v2)
    
    def evaluate(self):
        right_v1 = self.value1.evaluate() if type(self.value1) == EitherDiv else Right(self.value1)
        right_v2 = self.value2.evaluate() if type(self.value2) == EitherDiv else Right(self.value2)
        
        return right_v1 | (lambda v1: right_v2 | (lambda v2: self.div(v1,v2)))
        
    def __str__(self):
        return "EitherDiv(%s, %s)" % (self.value1, self.value2)
    

print(EitherDiv(10,EitherDiv(EitherDiv(2,1), 1)).evaluate())
print(EitherDiv(10,EitherDiv(EitherDiv(2,1), 0)).evaluate())

Right 5
Left (division by zero) EitherDiv(EitherDiv(2, 1), 0)


## Async Image fetching
Lets see the following code: It uses dog.ceo API to get random pictures of dogs. 

In [15]:
import requests
from IPython.display import Image 

def get_url(url):
    print("requesting %s" % url)
    return requests.get(url)

def fetch_image(url):
    print("generating image %s" % url)
    return Image(url=url)

href = get_url("https://dog.ceo/api/breeds/image/random").json()["message"]
image = fetch_image(href)
display(image)

href = get_url("https://dog.ceo/api/breeds/image/random").json()["message"]
image = fetch_image(href)
display(image)
    


requesting https://dog.ceo/api/breeds/image/random
generating image https://images.dog.ceo/breeds/pointer-german/n02100236_4235.jpg


requesting https://dog.ceo/api/breeds/image/random
generating image https://images.dog.ceo/breeds/terrier-kerryblue/n02093859_248.jpg


### I've decided to use asyncio Futures in Python. 

In [16]:
import asyncio
import requests
from IPython.display import Image 

def get_url(url):
    print("requesting %s" % url)
    return requests.get(url)

def fetch_image(url):
    print("generating image %s" % url)
    return Image(url=url)

def random_dog_callback(result_future):
    url = result_future.result().json()["message"]
    fututre_image = loop.run_in_executor(None, fetch_image, url)
    fututre_image.add_done_callback(lambda img: display(img.result()))

loop = asyncio.get_running_loop()
future_result = loop.run_in_executor(None, get_url, "https://dog.ceo/api/breeds/image/random")
print("waiting for random dog...")
future_result.add_done_callback(random_dog_callback)

future_result = loop.run_in_executor(None, get_url, "https://dog.ceo/api/breeds/image/random")
print("waiting for random dog...")
future_result.add_done_callback(random_dog_callback)


requesting https://dog.ceo/api/breeds/image/random
waiting for random dog...
requesting https://dog.ceo/api/breeds/image/randomwaiting for random dog...

generating image https://images.dog.ceo/breeds/airedale/n02096051_2353.jpg


generating image https://images.dog.ceo/breeds/sheepdog-shetland/n02105855_10095.jpg


## The Future doesn't look bright
Futures in Python don't have a very simple API, and working with asyncio Event Loop in python is no fun either. This code is not reusable, and has a lot of clutter just to get an image asyncronously. 

If we need to do it many times in our code base, in this constilation we will have to pad our code with get_running_loops and run_in_executor, and we also have a potential for callback hells.

We can write a Functor that will encapsulate the clutter, but our callbacks can be asyncronous as well. But, we can use the Monad Design Pattern.

## I promise you things will get better now...

Vows wrap the Future objects but simplifies its API while also hiding the over head of the asyncio Event Loop API. Lets implement that

In [17]:
loop = asyncio.get_running_loop()

def defer(func, delay=0):
    """
    runs func in event loop after `delay` of seconds
    returns a Vow
    """
    global loop
    #if delay == 0:
    #    return Vow(loop.run_in_executor(None, func))
    
    future = create_future()
    loop.call_later(delay, lambda: future.set_result(func()))
    return Vow(future)


def defer_func(func, delay=0):
    """
    returns a function that when executed will defer `func` after `delay` seconds
    """
    return lambda *args: defer(lambda: func(*args), delay)

def create_future():
    """
    creates a future and returns it
    """
    global loop
    return loop.create_future()


class Vow:
    def __init__(self, future):
        """
        wraps a future into a vow
        """
        if not asyncio.isfuture(future):
            raise Exception("Can't wrap non-futures")
        self.future = future
    
    def map(self, func):
        """
        runs `func` on the future result and returns a new vow with the result
        """
        new_future = create_future()
        self.future.add_done_callback(lambda _: new_future.set_result(func(self.future.result())))
        return self.__class__(new_future)
    
    def flat_map(self, func):
        """
        runs `func` on the future result and flatten it.
        returns flatten value is a Vow
        """
        new_future = create_future()
        self.future.add_done_callback(lambda _: func(self.future.result()).map(new_future.set_result))
        return self.__class__(new_future)
               
    def __str__(self):
        return ("VOW(%s)" % self.future)
    
    @classmethod
    def resolve(cls, value):
        """
        returns a resolved vow with `value`
        """
        if asyncio.isfuture(value):
            return cls(value)
        
        future = create_future()
        future.set_result(value)
        return cls(future)
    
    def when(self, func):
        """
        runs func on value - if it is a vow, it will flatten.
        the result is wrapped in a new vow.
        """
        def smart_map(_):
            result = func(self.future.result())
            if type(result) == Vow:
                result.map(new_future.set_result)
            else:
                new_future.set_result(result)
                
        new_future = create_future()
        self.future.add_done_callback(defer_func(smart_map))
        return self.__class__(new_future)
        
        



In [18]:



Vow.resolve("https://dog.ceo/api/breeds/image/random") \
    .flat_map(defer_func(get_url)) \
    .flat_map(lambda response: defer(response.json)) \
    .map(lambda json: json["message"]) \
    .flat_map(defer_func(fetch_image)) \
    .map(display)

print("")


requesting https://dog.ceo/api/breeds/image/random
generating image https://images.dog.ceo/breeds/terrier-wheaten/n02098105_1419.jpg


### We can do better

In [19]:
randoms = [Vow.resolve("https://dog.ceo/api/breeds/image/random"), Vow.resolve("https://dog.ceo/api/breeds/image/random")]
for r in randoms:
    r.when(get_url)\
    .when(lambda response: response.json()) \
    .when(lambda json: json["message"]) \
    .when(fetch_image) \
    .when(defer_func(lambda x:x, 2)) \
    .when(display)
print("")


requesting https://dog.ceo/api/breeds/image/random
requesting https://dog.ceo/api/breeds/image/random
generating image https://images.dog.ceo/breeds/spaniel-sussex/n02102480_576.jpg
generating image https://images.dog.ceo/breeds/terrier-westhighland/n02098286_238.jpg


In [21]:
def whened(func):
    def whened_func(self):
        result = self.when(lambda v: func(self, v))
        return result
    return whened_func

class ImageFetcher(Vow):
    @whened
    def get_url(self, url):
        print("requesting %s" % url)
        return requests.get(url)
    
    @whened
    def extract_image_url(self, response):
        json = response.json()
        return json["message"]

    @whened
    def fetch_image(self, url):
        print("generating image %s" % url)
        return Image(url=url)
    
    @whened
    def display(self, img):
        display(img)
    
        


In [22]:
images = [ 
    ImageFetcher.resolve("https://dog.ceo/api/breeds/image/random"),
    ImageFetcher.resolve("https://dog.ceo/api/breeds/image/random"),
    ImageFetcher.resolve("https://dog.ceo/api/breeds/image/random")
]

for image in images:
    image.get_url().extract_image_url().fetch_image().display()
print("")


requesting https://dog.ceo/api/breeds/image/random
requesting https://dog.ceo/api/breeds/image/random
requesting https://dog.ceo/api/breeds/image/random
generating image https://images.dog.ceo/breeds/chow/n02112137_4760.jpg
generating image https://images.dog.ceo/breeds/terrier-dandie/n02096437_2055.jpg
generating image https://images.dog.ceo/breeds/poodle-toy/n02113624_2609.jpg
